In [1]:
pip install google-cloud-bigquery

In [2]:
from google.cloud import bigquery
import os
import pandas as pd
import datetime as dt
import numpy as np
from pandas.io import gbq

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd "/content/drive/Shareddrives/Tieu_Quyen/Data_Science/Data Engineering/DE2"

/content/drive/Shareddrives/Tieu_Quyen/Data_Science/Data Engineering/DE2


## Data collection API

In [5]:
import requests
import json
from requests.auth import HTTPBasicAuth
import pandas as pd
from pandas.io.json import json_normalize

In [6]:
#: API key
keyVal='70f735fa7557ad81fe3550508d8bb522'
auth = HTTPBasicAuth(keyVal, '')

In [7]:
# Trending media_type
api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page=1&with_watch_monetization_types=flatrate'
data=requests.get(api_endpoint, auth=auth).json()

In [9]:
# Find out total number of pages
api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page=1&with_watch_monetization_types=flatrate'
data = requests.get(url=api_endpoint,auth=auth).json()
total_pages = data['total_pages']
total_pages

500

In [10]:
data = pd.DataFrame([])
for page in range (1, total_pages):
    api_endpoint=f'https://api.themoviedb.org/3/discover/movie?api_key={keyVal}&language=en-US&sort_by=popularity.desc&include_adult=true&include_video=true&page={page}&with_watch_monetization_types=flatrate'
    df=requests.get(api_endpoint, auth=auth).json()
    df_new = pd.json_normalize(df['results'])
    data = pd.concat([data, df_new])
    page += 1

In [11]:
data.count()

adult                9980
backdrop_path        9578
genre_ids            9980
id                   9980
original_language    9980
original_title       9980
overview             9980
popularity           9980
poster_path          9940
release_date         9970
title                9980
video                9980
vote_average         9980
vote_count           9980
dtype: int64

## Data Pre-processing

In [12]:
movie = data[['adult','id','original_language','overview','popularity','release_date','title','vote_average','vote_count']]
movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9980 entries, 0 to 19
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              9980 non-null   bool   
 1   id                 9980 non-null   int64  
 2   original_language  9980 non-null   object 
 3   overview           9980 non-null   object 
 4   popularity         9980 non-null   float64
 5   release_date       9970 non-null   object 
 6   title              9980 non-null   object 
 7   vote_average       9980 non-null   float64
 8   vote_count         9980 non-null   int64  
dtypes: bool(1), float64(2), int64(2), object(4)
memory usage: 711.5+ KB


In [13]:
movie['year'] = pd.to_datetime(movie['release_date']).dt.year
movie.head()


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year
0,False,580489,en,After finding a host body in investigative rep...,8633.976,2021-09-30,Venom: Let There Be Carnage,7.2,4398,2021.0
1,False,512195,en,An Interpol-issued Red Notice is a global aler...,4825.133,2021-11-04,Red Notice,6.8,1970,2021.0
2,False,585245,en,As Emily struggles to fit in at home and at sc...,3280.160,2021-11-10,Clifford the Big Red Dog,7.5,530,2021.0
3,False,617653,en,King Charles VI declares that Knight Jean de C...,3256.364,2021-10-13,The Last Duel,7.6,870,2021.0
4,False,566525,en,Shang-Chi must confront the past he thought he...,3189.442,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4201,2021.0


In [14]:
movie['month'] = pd.to_datetime(movie['release_date']).dt.month
movie.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year,month
0,False,580489,en,After finding a host body in investigative rep...,8633.976,2021-09-30,Venom: Let There Be Carnage,7.2,4398,2021.0,9.0
1,False,512195,en,An Interpol-issued Red Notice is a global aler...,4825.133,2021-11-04,Red Notice,6.8,1970,2021.0,11.0
2,False,585245,en,As Emily struggles to fit in at home and at sc...,3280.160,2021-11-10,Clifford the Big Red Dog,7.5,530,2021.0,11.0
3,False,617653,en,King Charles VI declares that Knight Jean de C...,3256.364,2021-10-13,The Last Duel,7.6,870,2021.0,10.0
4,False,566525,en,Shang-Chi must confront the past he thought he...,3189.442,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4201,2021.0,9.0


In [15]:
movie_list = movie.where((movie['original_language']=='en') & (movie['year']==2021) & (movie['vote_count']>0))
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 9980 entries, 0 to 19
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              513 non-null    float64
 1   id                 513 non-null    float64
 2   original_language  513 non-null    object 
 3   overview           513 non-null    object 
 4   popularity         513 non-null    float64
 5   release_date       513 non-null    object 
 6   title              513 non-null    object 
 7   vote_average       513 non-null    float64
 8   vote_count         513 non-null    float64
 9   year               513 non-null    float64
 10  month              513 non-null    float64
dtypes: float64(7), object(4)
memory usage: 935.6+ KB


In [16]:
movie_list=movie_list.dropna(how='any') 
movie_list.head()

,adult,id,original_language,overview,popularity,release_date,title,vote_average,vote_count,year,month
0,0.0,580489.0,en,After finding a host body in investigative rep...,8633.976,2021-09-30,Venom: Let There Be Carnage,7.2,4398.0,2021.0,9.0
1,0.0,512195.0,en,An Interpol-issued Red Notice is a global aler...,4825.133,2021-11-04,Red Notice,6.8,1970.0,2021.0,11.0
2,0.0,585245.0,en,As Emily struggles to fit in at home and at sc...,3280.160,2021-11-10,Clifford the Big Red Dog,7.5,530.0,2021.0,11.0
3,0.0,617653.0,en,King Charles VI declares that Knight Jean de C...,3256.364,2021-10-13,The Last Duel,7.6,870.0,2021.0,10.0
4,0.0,566525.0,en,Shang-Chi must confront the past he thought he...,3189.442,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4201.0,2021.0,9.0


In [17]:
movie_list=movie_list.drop_duplicates()
movie_list.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 0 to 4
Data columns (total 11 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   adult              513 non-null    float64
 1   id                 513 non-null    float64
 2   original_language  513 non-null    object 
 3   overview           513 non-null    object 
 4   popularity         513 non-null    float64
 5   release_date       513 non-null    object 
 6   title              513 non-null    object 
 7   vote_average       513 non-null    float64
 8   vote_count         513 non-null    float64
 9   year               513 non-null    float64
 10  month              513 non-null    float64
dtypes: float64(7), object(4)
memory usage: 48.1+ KB


In [18]:
final_movie = movie_list[['adult','id','overview','popularity','release_date','title','vote_average','vote_count']]
final_movie.head()

,adult,id,overview,popularity,release_date,title,vote_average,vote_count
0,0.0,580489.0,After finding a host body in investigative rep...,8633.976,2021-09-30,Venom: Let There Be Carnage,7.2,4398.0
1,0.0,512195.0,An Interpol-issued Red Notice is a global aler...,4825.133,2021-11-04,Red Notice,6.8,1970.0
2,0.0,585245.0,As Emily struggles to fit in at home and at sc...,3280.160,2021-11-10,Clifford the Big Red Dog,7.5,530.0
3,0.0,617653.0,King Charles VI declares that Knight Jean de C...,3256.364,2021-10-13,The Last Duel,7.6,870.0
4,0.0,566525.0,Shang-Chi must confront the past he thought he...,3189.442,2021-09-01,Shang-Chi and the Legend of the Ten Rings,7.8,4201.0


In [19]:
final_movie.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 513 entries, 0 to 4
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   adult         513 non-null    float64
 1   id            513 non-null    float64
 2   overview      513 non-null    object 
 3   popularity    513 non-null    float64
 4   release_date  513 non-null    object 
 5   title         513 non-null    object 
 6   vote_average  513 non-null    float64
 7   vote_count    513 non-null    float64
dtypes: float64(5), object(3)
memory usage: 36.1+ KB


## Load data to GCP

In [ ]:
final_movie.to_gbq(destination_table='movie_dataset.realtime',
                   project_id='direct-obelisk-333711',
                   if_exists='replace')

1it [00:04,  4.02s/it]
